In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mne # reads edf format
import glob
import random
import re
import wfdb # waveform database library wfdb.rdann reads annotation of physiology annotated data in ECG, EEG etc
import tqdm # time bar

In [2]:
channel_labels = ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3','P3-O1',
                  'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2',
                  'FZ-CZ', 'CZ-PZ']

In [3]:
path = r'data\raw_data\chb-mit-scalp-eeg-database-1.0.0'
data_folders = sorted(glob.glob(os.path.join(path, '*[0-9]')))

def file_id(folder):
    return [ name[-2:] for name in [file.rsplit('\\',2)[-1] for file in folder]]
print("ID: ")
print(file_id(data_folders))

ID: 
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']


In [4]:
# seizure  marking is in name of edf file, not a separate file
# split files for training and testing
train_test_ratio = 0.8
random.seed(80)
train_folders = sorted(random.sample(data_folders, round(len(data_folders) * train_test_ratio)))
test_folders = sorted([ file for file in data_folders if file not in train_folders])
print(f"Training files ID: {file_id(train_folders)}")
print(f"Test files ID: {file_id(test_folders)}")



Training files ID: ['01', '02', '04', '07', '08', '09', '10', '12', '13', '14', '15', '16', '17', '18', '19', '20', '22', '23', '24']
Test files ID: ['03', '05', '06', '11', '21']


In [5]:
# Retrieve edf files

train_files = [ file for folder in train_folders for file in glob.glob(folder+'/*.edf')]
test_files = [ file for folder in test_folders for file in glob.glob(folder+'/*.edf')]

print(f"Train_files contains {len(train_files)} files")
print(f"Test_files contains {len(test_files)} files")

Train_files contains 523 files
Test_files contains 163 files


In [6]:
# MEG and EEG library (mne) reads neurophysiological data
sample_edf = mne.io.read_raw_edf(train_files[0], preload=False) #110-15 seconds
# dir(sample_edf)  # show all attributes and methods
sample_edf.info

Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\mspla\AppData\Local\Temp\ipykernel_11760\922965260.py:2: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  sample_edf = mne.io.read_raw_edf(train_files[0], preload=False) #110-15 seconds


<Info | 8 non-empty values
 bads: []
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2076-11-06 11:42:54 UTC
 nchan: 23
 projs: []
 sfreq: 256.0 Hz
 subject_info: 1 item (dict)
>

In [7]:
'''
Reading EEG data, modified from cell 60 of notebook by Masahiro Gotoh   [available at https://www.kaggle.com/code/hemangjindal/chb-mit-eeg-dataset-my-notebook#CHB-MIT-eeg-dataset-seizure-detection-demo]
'''
# constants
WINDOW_TIME = 8  # segment size in second
STEP_TIME = 4     # Step size in second
SAMPLING_RATE = 16  # EEG Sampling rate (Hz)
SEIZURE_PROPORTION = 0.01    # proportion of non seizure, data is imbalanced with less than 1% comprising seizure data
TO_MICROVOLTS = 1e6

def read_edf(folder):
    count=0
    window_size=0

    for file in folder:
        edf_data = mne.io.read_raw_edf(file, preload=False)
        edf_labels = edf_data.ch_names
        # check all channel labels appear in edf_labels
        if sum([any([0 if re.match(c, l) is None else 1 for l in edf_labels]) for c in channel_labels]) == len(channel_labels):
            sampling_freq = int(1/(edf_data.times[1]-edf_data.times[0]))
            window_size = sampling_freq * WINDOW_TIME
            window_stride = sampling_freq * STEP_TIME

            # identity EEG signal with seizure. 'Seizure' appended to file name, and marked at time point of seizure activity inside file.
            # has seizure marks seizure/non-seizure as (1/0) for every data point
            has_seizure = np.zeros((edf_data.n_times,))
            if os.path.exists(file + '.seizures'):
                has_annotation = wfdb.rdann(file, 'seizures')
                # has_annotation.sample e.g [10,20, 300,400] means marked seizure from sample 10-20 , and 300-400
                for idx in range(int(has_annotation.sample.size / 2)):
                    has_seizure[has_annotation.sample[idx * 2]:has_annotation.sample[idx * 2 + 1]] = 1

            #creawte seizure index, and calculate proportion of signal which shows seizures
            is_seizure_idx = np.array([has_seizure[idx * window_stride:idx * window_stride + window_size].sum() / window_size for idx in range((edf_data.n_times - window_size) // window_stride)])

            ### size of samples with and without seizure after subsampling
            ### Data imbalance, normal EEG comprises 99% of data, multiplying by no seizure portion 0.01% allows downsampling of non seizure data
            noseizure_n_size = round(SEIZURE_PROPORTION * np.where(is_seizure_idx==0)[0].size)
            seizure_n_size = np.where(is_seizure_idx > 0)[0].size
            count = count + noseizure_n_size + seizure_n_size
        edf_data.close()

    #initialise
    signals_np = np.zeros((count, len(channel_labels), window_size), dtype=np.float32)
    labels_seizure_np = np.zeros(count, dtype=bool)
    return count, len(channel_labels), window_size

In [16]:
9
def extract_edf2(folder, n_samples, n_channel_labels, window_size):
    if os.path.exists("eeg_signals.npy") and os.path.exists("seizure_labels.npy"):
        print("File eeg_signals.npy and seizure_labels.npy exists.")
        return
    signals_np = np.zeros((n_samples, n_channel_labels, window_size), dtype=np.float32)
    labels_np = np.zeros(n_samples, dtype=bool)


    # Read  edf, rename file channel to match names from channel labels list. When files have multiple channel names, only first is picked.
    for number, file in enumerate(tqdm.tqdm(folder)):
        log = f"Reading file {number} "
        edf_file = mne.io.read_raw_edf(file, preload=False)

        n_label_match = sum([any([0 if re.match(ch, ch_name) is None else 1 for ch_name in edf_file.ch_names]) for ch in channel_labels])
        if n_label_match == len(channel_labels):
            dict_ch_name = {sorted([ch_name for ch_name in edf_file.ch_names if re.match(ch, ch_name) is not None])[0]: ch for ch in channel_labels}
            edf_file.rename_channels(dict_ch_name)

            # Retrieve EEG (in microvolts) ,  annotations
            has_seizure = np.zeros((edf_file.n_times,))
            signals_ = edf_file.get_data(picks=channel_labels) * TO_MICROVOLTS

            if os.path.exists(file + '.seizures'):
                log += "positive seizure"
                has_annotation = wfdb.rdann(file, 'seizures')
                for idx in range(int(has_annotation.sample.size / 2)):
                    has_seizure[has_annotation.sample[idx * 2]:has_annotation.sample[idx * 2 + 1]] = 1
            else:
                log += "negative seizure"

            # create seizure index, and calculate proportion of signal which shows seizures
            sampling_freq = int(1/(edf_file.times[1] - edf_file.times[0]))
            print("sampling frequency ",sampling_freq)
            window_stride = int(sampling_freq * STEP_TIME)
            print(f"Window stride: {window_stride}, Window size: {window_size}")
            is_seizure_idx = np.array([has_seizure[idx * window_stride:idx * window_stride + window_size].sum() / window_size for idx in range((edf_file.n_times - window_size) // window_stride)])

            # populate numpy array with EEG , annotation data
            noseizure_n_size = round(SEIZURE_PROPORTION * np.where(is_seizure_idx == 0)[0].size)
            seizure_n_size = np.where(is_seizure_idx > 0)[0].size


            # non seizure data are by far larger than seizure data. To avoid overfitting, and bias, non seizure data is randomly subsampled. This prevents model from being overwhelmed by large non seizure data
            count = 0
            # no seizure
            temp_negative = random.sample(list(np.where(is_seizure_idx == 0)[0]), noseizure_n_size)
            for value in temp_negative:
                start_idx = value * window_stride
                end_idx = start_idx + window_size
                segment = signals_[:, start_idx:end_idx]
                print(f"Segment shape: {segment.shape}")  # Debug print
                if segment.shape == (n_channel_labels, window_size):
                    signals_np[count, :, :] = segment
                    labels_np[count] = False
                    count += 1
                else:
                    print(f"Skipping segment due to shape mismatch: {segment.shape}")

            # seizure
            temp_positive = list(np.where(is_seizure_idx > 0)[0])
            for value in temp_positive:
                start_idx = value * window_stride
                end_idx = start_idx + window_size
                segment = signals_[:, start_idx:end_idx]
                print(f"Segment shape: {segment.shape}")  # Debug print
                if segment.shape == (n_channel_labels, window_size):
                    signals_np[count, :, :] = segment
                    labels_np[count] = True
                    count += 1
                else:
                    print(f"Skipping segment due to shape mismatch: {segment.shape}")

            print(f"{noseizure_n_size + seizure_n_size} EEG signals added {seizure_n_size} with, {noseizure_n_size} without seizures")
        else:
            print(f"Unable to read {file}")

        edf_file.close()

    np.save('eeg_signals', signals_np)
    np.save('seizure_labels', labels_np)



In [ ]:
sample_length, channel_length, window_length = read_edf(train_files)

In [14]:
#extract_edf(train_files, sample_length, channel_length, window_length)
extract_edf2(train_files, sample_length, channel_length, window_length)

  0%|          | 0/523 [00:00<?, ?it/s]

Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\mspla\AppData\Local\Temp\ipykernel_11760\1130633979.py:8: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_file = mne.io.read_raw_edf(file, preload=False)
  0%|          | 0/523 [00:00<?, ?it/s]

sampling frequency  256
42
signals_np (9529, 18, 2048)
signals_ (18, 921600)


ValueError: could not broadcast input array from shape (18,0) into shape (18,2048)

In [13]:
def extract_edf(folder, n_samples, n_channel_labels, window_size):
    signals_np = np.zeros((n_samples, n_channel_labels, window_size), dtype=np.float32)
    labels_np = np.zeros(n_samples, dtype=bool)

    # Read  edf, rename file channel to match names from channel labels list. When files have multiple channel names, only first is picked.
    for number, file in enumerate(tqdm.tqdm(folder)):
        log = f"Reading file {number} "
        edf_file = mne.io.read_raw_edf(file, preload=False)

        n_label_match = sum([any([0 if re.match(ch, ch_name) is None else 1 for ch_name in edf_file.ch_names]) for ch in channel_labels])
        if n_label_match == len(channel_labels):
            dict_ch_name = {sorted([ch_name for ch_name in edf_file.ch_names if re.match(ch, ch_name) is not None])[0]: ch for ch in channel_labels}
            edf_file.rename_channels(dict_ch_name)

            # Retrieve EEG (in microvolts) ,  annotations
            has_seizure = np.zeros((edf_file.n_times,))
            signals_ = edf_file.get_data(picks=channel_labels) * TO_MICROVOLTS

            if  os.path.exists(file+'.seizures'):
                log = log + "positive seizure"
                has_annotation = wfdb.rdann(file, 'seizures')
                for idx in range(int(has_annotation.sample.size / 2)):
                    has_seizure[has_annotation.sample[idx * 2]:has_annotation.sample[idx * 2 + 1]] = 1
            else:
                log = log + "negative seizure"

            # create seizure index, and calculate proportion of signal which shows seizures
            sampling_freq = int(1/(edf_file.times[1]-edf_file.times[0]))
            print("sampling frequency ",sampling_freq)
            window_stride = int(sampling_freq * STEP_TIME)
            is_seizure_idx = np.array([has_seizure[idx * window_stride:idx * window_stride + window_size].sum() / window_size for idx in range((edf_file.n_times - window_size) // window_stride)])

            # populate numpy array with EEG , annotation data
            noseizure_n_size = round(SEIZURE_PROPORTION * np.where(is_seizure_idx==0)[0].size)
            seizure_n_size = np.where(is_seizure_idx > 0)[0].size


            # non seizure data are by far larger than seizure data. To avoid overfitting, and bias, non seizure data is randomly subsampled. This prevents model
            # from being overwhelmed by large non seizure data
            count = 0
            temp_negative = random.sample(list(np.where(is_seizure_idx == 0)[0]), noseizure_n_size)
            for value in temp_negative:
                print("42")
                print("signals_np",signals_np.shape)
                print("signals_",signals_.shape)
                start_index = value * window_size
                stop_index = value * window_stride + window_size
                signals_np[count, :, :] = signals_[:, start_index:stop_index ]
                print("45")
                labels_np[count] = False
                count = count + 1
            #seizure

            temp_positive = list(np.where(is_seizure_idx == 1)[0])
            for value in temp_positive:
                start_index = value * window_stride
                stop_index = value * window_stride + window_size
                signals_np[count, :, :] = signals_[:, start_index: stop_index]
                labels_np[count] = True
                count = count + 1

            print(f"{noseizure_n_size+seizure_n_size} EEG signals added {seizure_n_size} with, {noseizure_n_size} without seizures")
        else:
            print(f"Unable to read {file}")

        # close resource
        edf_file.close()

        # save signal and label files
    np.save('eeg_signals', signals_np)
    np.save('seizure_labels', labels_np)



In [19]:
path_signal = 'eeg_signals.npy'
path_label = 'seizure_labels.npy'
data_signals = np.load(path_signal)
data_labels = np.load(path_label)

In [20]:
data_signals.shape,data_labels.shape

((9529, 18, 2048), (9529,))

In [21]:
n_samples, n_channels, segment_length = data_signals.shape
n_labels = data_labels.shape[0]

In [22]:
print(f"Numpy file {path_signal:<19} has {n_samples:<6} samples with {n_channels:} channels, and segment length of {segment_length}")
print(f"Numpy file {path_label:<19} has {n_labels:<6} labels for seizure presence or absence.")

Numpy file eeg_signals.npy     has 9529   samples with 18 channels, and segment length of 2048
Numpy file seizure_labels.npy  has 9529   labels for seizure presence or absence.


In [ ]:
# END

In [ ]:

'''

# Configuration
csv_folder = '/kaggle/working/csv_files'  # Folder containing the merged CSV file
# eeg_channels = ['Channel_1', 'Channel_13', 'Channel_19', 'Channel_23']  # Channels to use
eeg_channels = ['Channel_5', 'Channel_15', 'Channel_3', 'Channel_2']  # Channels to use

window_size_sec = 5  # Window size in seconds
step_size_sec = 2 # Step size in seconds
sampling_rate = 16  # Sampling rate of EEG signals in Hz
preictal_label = 1
interictal_label = 0

# Derived parameters
window_size = window_size_sec * sampling_rate  # Convert to samples
step_size = step_size_sec * sampling_rate  # Convert to samples

# Function to load and prepare data
def load_and_prepare_data(csv_file):
    """
    Load the merged CSV data and prepare it for LSTM model training.
    """
    data = pd.read_csv(csv_file)

    # Normalize the EEG data (Min-Max scaling)
    scaler = MinMaxScaler()
    eeg_data = data[eeg_channels].values
    eeg_data_scaled = scaler.fit_transform(eeg_data)

    # Get the labels
    labels = data['Label'].values

    # Create sliding windows
    windows, window_labels = create_windows(eeg_data_scaled, labels, window_size, step_size)

    return windows, window_labels, scaler

# Function to create windows from the data
def create_windows(data, labels, window_size, step_size):
    """
    Create overlapping windows from EEG data and corresponding labels.
    """
    windows = []
    window_labels = []

    for start in range(0, len(data) - window_size + 1, step_size):
        end = start + window_size
        window = data[start:end, :]
        label = labels[start:end]

        # Assign label based on majority class in the window
        window_label = preictal_label if np.sum(label == preictal_label) > len(label) // 2 else interictal_label
        windows.append(window)
        window_labels.append(window_label)

    return np.array(windows), np.array(window_labels)

# Function to build the LSTM model
def build_lstm_model(input_shape):
    """
    Build and compile an LSTM model.
    """
    print('Input shape-',input_shape)
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid for binary classification
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to train the LSTM model
def train_lstm_model(windows, window_labels):
    """
    Train the LSTM model using the provided windows and labels.
    """
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(windows, window_labels, test_size=0.2, random_state=42)

    # Build and train the model
    model = build_lstm_model((X_train.shape[1], X_train.shape[2]))
    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary labels

    # Print classification report
    print("Classification Report on Test Data:")
    print(classification_report(y_test, y_pred))

    # Return the trained model
    return model, scaler

# Real-time prediction function
def real_time_prediction(model, scaler, window_size, interval=30):
    """
    Simulate real-time predictions every `interval` seconds using the trained LSTM model.
    """
    print(f"Starting real-time prediction... (Interval: {interval}s)")

    # Simulate receiving new data (use the merged data as a placeholder)
    while True:
        # Simulate receiving data
        # Replace this with real-time data fetching process
        # For now, we assume the data is available in 'merged_df'
        # Here we can simulate a chunk of data for prediction
        simulated_data = np.random.rand(window_size, len(eeg_channels))  # Simulating new data

        # Normalize the new data
        scaled_data = scaler.transform(simulated_data)

        # Reshape data to match LSTM input shape
        X_input = scaled_data.reshape(1, window_size, len(eeg_channels))

        # Predict the probability of seizure (preictal)
        prediction = model.predict(X_input)
        print(f"Predicted probability of seizure: {prediction[0][0]:.4f}")

        # Sleep for the specified interval (simulate 30 seconds)
        time.sleep(interval)

# Main function
if __name__ == "__main__":
    # Load the merged data
    csv_file = '/kaggle/working/csv_files/merged_data.csv'
    windows, window_labels, scaler = load_and_prepare_data(csv_file)

    # Train the LSTM model and evaluate on the test data
    model, scaler = train_lstm_model(windows, window_labels)

    # Save the model for later use
    model.save('/kaggle/working/lstm_model.h5')
    print("Model saved to /kaggle/working/lstm_model.h5")

    # Start real-time prediction
    real_time_prediction(model, scaler, window_size, interval=30)

'''

In [21]:
def read_edf(files_train):
    time_window = 8
    time_step = 4

    if os.path.exists('/kaggle/input/mit-chb-processed/signal_samples.npy')&os.path.exists('/kaggle/input/mit-chb-processed/is_sz.npy'):
        array_signals=np.load('/kaggle/input/mit-chb-processed/signal_samples.npy')
        array_is_sz=np.load('/kaggle/input/mit-chb-processed/is_sz.npy')
    else:
        p = 0.01
        counter = 0
        step_window = 0
        for temp_f in files_train:
            temp_edf =  mne.io.read_raw_edf(temp_f)
            temp_labels = temp_edf.ch_names
            if sum([any([0 if re.match(c, l) is None else 1 for l in temp_edf.ch_names]) for c in ch_labels])==len(ch_labels):
                time_window = 8
                time_step = 4
                fs = int(1/(temp_edf.times[1]-temp_edf.times[0]))
                step_window = time_window*fs
                step = time_step*fs

                temp_is_sz = np.zeros((temp_edf.n_times,))
                if os.path.exists(temp_f+'.seizures'):
                    temp_annotation = wfdb.rdann(temp_f, 'seizures')
                    for i in range(int(temp_annotation.sample.size/2)):
                        temp_is_sz[temp_annotation.sample[i*2]:temp_annotation.sample[i*2+1]]=1
                temp_len = temp_edf.n_times

                temp_is_sz_ind = np.array(
                    [temp_is_sz[i*step:i*step+step_window].sum()/step_window for i in range((temp_len-step_window)//step)]
                )

                temp_0_sample_size = round(p*np.where(temp_is_sz_ind==0)[0].size)
                temp_1_sample_size = np.where(temp_is_sz_ind>0)[0].size

                counter = counter + temp_0_sample_size + temp_1_sample_size
            temp_edf.close()
            return counter, ch_labels, step_window

def extract_edf2(files_train, counter, ch_labels, step_window):
        array_signals = np.zeros((counter, len(ch_labels), step_window), dtype=np.float32)
        array_is_sz = np.zeros(counter, dtype=bool)

        counter = 0
        for n, temp_f in enumerate(tqdm.tqdm(files_train)):
            to_log = 'No. {}: Reading. '.format(n)
            temp_edf =  mne.io.read_raw_edf(temp_f)
            temp_labels = temp_edf.ch_names
            n_label_match = sum([any([0 if re.match(c, l)==None else 1 for l in temp_edf.ch_names]) for c in ch_labels])
            if n_label_match==len(ch_labels):
                ch_mapping = {sorted([l for l in temp_edf.ch_names if re.match(c, l)!=None ])[0]:c for c in ch_labels}
                temp_edf.rename_channels(ch_mapping)
                #temp_edf = temp_edf.pick(ch_labels)

                temp_is_sz = np.zeros((temp_edf.n_times,))
                temp_signals = temp_edf.get_data(picks=ch_labels)*1e6

                if os.path.exists(temp_f+'.seizures'):
                    to_log = to_log+'sz exists.'
                    temp_annotation = wfdb.rdann(temp_f, 'seizures')
                    for i in range(int(temp_annotation.sample.size/2)):
                        temp_is_sz[temp_annotation.sample[i*2]:temp_annotation.sample[i*2+1]]=1
                else:
                    to_log = to_log+'No sz.'

                temp_len = temp_edf.n_times

                time_window = 8
                time_step = 4
                fs = int(1/(temp_edf.times[1]-temp_edf.times[0]))
                step_window = time_window*fs
                step = time_step*fs

                temp_is_sz_ind = np.array(
                    [temp_is_sz[i*step:i*step+step_window].sum()/step_window for i in range((temp_len-step_window)//step)]
                )
                del temp_is_sz

                temp_0_sample_size = round(p*np.where(temp_is_sz_ind==0)[0].size)
                temp_1_sample_size = np.where(temp_is_sz_ind>0)[0].size

                # sz data
                temp_ind = list(np.where(temp_is_sz_ind>0)[0])
                for i in temp_ind:
                    array_signals[counter, :, :] = temp_signals[:, i*step:i*step+step_window]
                    array_is_sz[counter] = True
                    counter = counter+1

                # no sz data
                temp_ind = random.sample(list(np.where(temp_is_sz_ind==0)[0]), temp_0_sample_size)
                for i in temp_ind:
                    array_signals[counter, :, :] = temp_signals[:, i*step:i*step+step_window]
                    array_is_sz[counter] = False
                    counter = counter+1

                to_log += '{} signals added: {} w/o sz, {} w/ sz.'.format(
                    temp_0_sample_size+temp_1_sample_size, temp_0_sample_size, temp_1_sample_size
                )
                print("array_signals",array_signals.shape, array_signals[:2], "temp_signals",temp_signals.shape, temp_signals[:2])

            else:
                to_log += 'Not appropriate channel labels. Reading skipped.'.format(n)


            temp_edf.close()

        np.save('signal_samples', array_signals)
        np.save('is_sz', array_is_sz)


In [22]:
count1, ch_labels1, step_window1 = read_edf(train_files)
count1, channel_labels,step_window1
#extract_edf2(train_files, count1, ch_labels1, step_window1)

Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\mspla\AppData\Local\Temp\ipykernel_6516\519428527.py:13: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  temp_edf =  mne.io.read_raw_edf(temp_f)


(9,
 ['FP1-F7',
  'F7-T7',
  'T7-P7',
  'P7-O1',
  'FP1-F3',
  'F3-C3',
  'C3-P3',
  'P3-O1',
  'FP2-F4',
  'F4-C4',
  'C4-P4',
  'P4-O2',
  'FP2-F8',
  'F8-T8',
  'T8-P8',
  'P8-O2',
  'FZ-CZ',
  'CZ-PZ'],
 2048)

In [16]:
ch_labels = ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3','P3-O1',
                  'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2',
                  'FZ-CZ', 'CZ-PZ']

In [ ]:
test(train_files)